### Lab2 (a) Model preperation

In [5]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np
import copy
import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [7]:
# Load the best weight paramters
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9151


### Lab2 (b) Prune by percentage

In [8]:
def prune_by_percentage(layer, q=80.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # Convert the weight of "layer" to numpy array
    out = layer.weight
    weight = out.cpu().detach().numpy()
    size=weight.shape
    weight_re = weight.reshape(-1,1)
    
    
    # Compute the q-th percentile of the abs of the converted array
    weight_abs = np.abs(weight_re)
    N = weight_abs.shape[0]
    index = np.argsort(weight_abs,axis=0)
    N_drop = int(np.ceil(N*(q*0.01)))
    index_drop = index[0:N_drop]
    
    # Generate a binary mask same shape as weight to decide which element to prune
    mask = np.ones((N,1))
    mask[index_drop] = 0
    mask =mask.reshape(size)

    #weight_new = np.multiply(mask, weight_re)
    #weight_output = weight_new.reshape(size)
    
    # Convert mask to torch tensor and put on GPU
    mask = torch.from_numpy(mask).to(layer.weight.device)

    
    # Multiply the weight by mask to perform pruning
    output = torch.mul(mask, out)
    layer.weight.data = output.float()
    
    pass
    

In [9]:
net.load_state_dict(torch.load("pretrained_model.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # change q value
        prune_by_percentage(layer, q=40.0)
        
        # Optional: Check the sparsity you achieve in each layer
        # Convert the weight of "layer" to numpy array
        out = layer.weight
        weight = out.cpu().detach().numpy()
        weight_re = weight.reshape(-1,1)
        
        weight_abs = np.abs(weight_re)
        N = weight_abs.shape[0]
        
        #np_weight = 
        # Count number of zeros
        zeros = N- np.count_nonzero(weight_abs)
        #zeros = 
        # Count number of parameters
        #total = 
        total = weight_abs.shape[0]
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.40046296296296297
Sparsity of body_op.0.conv1.0.conv: 0.4001736111111111
Sparsity of body_op.0.conv2.0.conv: 0.4001736111111111
Sparsity of body_op.1.conv1.0.conv: 0.4001736111111111
Sparsity of body_op.1.conv2.0.conv: 0.4001736111111111
Sparsity of body_op.2.conv1.0.conv: 0.4001736111111111
Sparsity of body_op.2.conv2.0.conv: 0.4001736111111111
Sparsity of body_op.3.conv1.0.conv: 0.4001736111111111
Sparsity of body_op.3.conv2.0.conv: 0.4000651041666667
Sparsity of body_op.4.conv1.0.conv: 0.4000651041666667
Sparsity of body_op.4.conv2.0.conv: 0.4000651041666667
Sparsity of body_op.5.conv1.0.conv: 0.4000651041666667
Sparsity of body_op.5.conv2.0.conv: 0.4000651041666667
Sparsity of body_op.6.conv1.0.conv: 0.4000108506944444
Sparsity of body_op.6.conv2.0.conv: 0.4000108506944444
Sparsity of body_op.7.conv1.0.conv: 0.4000108506944444
Sparsity of body_op.7.conv2.0.conv: 0.4000108506944444
Sparsity of body_op.8.conv1.0.conv: 0.4000108506944444
Sparsity of bod

In [10]:
for item in [40,60,80]:
    net.load_state_dict(torch.load("pretrained_model.pt"))

    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # change q value
            prune_by_percentage(layer, q=item)

            # Optional: Check the sparsity you achieve in each layer
            # Convert the weight of "layer" to numpy array
            out = layer.weight
            weight = out.cpu().detach().numpy()
            weight_re = weight.reshape(-1,1)

            weight_abs = np.abs(weight_re)
            N = weight_abs.shape[0]

            #np_weight = 
            # Count number of zeros
            zeros = N- np.count_nonzero(weight_abs)
            #zeros = 
            # Count number of parameters
            #total = 
            total = weight_abs.shape[0]
            # Print sparsity
            print('Sparsity of '+name+': '+str(zeros/total))
    test(net)
        

Sparsity of head_conv.0.conv: 0.40046296296296297
Sparsity of body_op.0.conv1.0.conv: 0.4001736111111111
Sparsity of body_op.0.conv2.0.conv: 0.4001736111111111
Sparsity of body_op.1.conv1.0.conv: 0.4001736111111111
Sparsity of body_op.1.conv2.0.conv: 0.4001736111111111
Sparsity of body_op.2.conv1.0.conv: 0.4001736111111111
Sparsity of body_op.2.conv2.0.conv: 0.4001736111111111
Sparsity of body_op.3.conv1.0.conv: 0.4001736111111111
Sparsity of body_op.3.conv2.0.conv: 0.4000651041666667
Sparsity of body_op.4.conv1.0.conv: 0.4000651041666667
Sparsity of body_op.4.conv2.0.conv: 0.4000651041666667
Sparsity of body_op.5.conv1.0.conv: 0.4000651041666667
Sparsity of body_op.5.conv2.0.conv: 0.4000651041666667
Sparsity of body_op.6.conv1.0.conv: 0.4000108506944444
Sparsity of body_op.6.conv2.0.conv: 0.4000108506944444
Sparsity of body_op.7.conv1.0.conv: 0.4000108506944444
Sparsity of body_op.7.conv2.0.conv: 0.4000108506944444
Sparsity of body_op.8.conv1.0.conv: 0.4000108506944444
Sparsity of bod

KeyboardInterrupt: 

### Lab2 (c) Finetune pruned model

In [9]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element 
            out = layer.weight
            weight = out.clone()
            weight[weight!=0] = 1      
            weight_mask[name] = weight
    
    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        if prune:
            for name,layer in net.named_modules():
                if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero
                    weight_id_mask = weight_mask[name]
                    weight_id_layer=layer.weight
                    layer.weight.data = torch.mul(weight_id_mask ,weight_id_layer)
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            num_examples_per_second = 50 * batch_size / (end - start)
            print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                 % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
            start = time.time()
    
    

In [12]:
# Get pruned model
net.load_state_dict(torch.load("pretrained_model.pt"))
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        prune_by_percentage(layer, q=80.0)

# Training setup, do not change
batch_size=256
lr=0.002
reg=1e-4

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [57]:
# Model finetuning
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer)
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "net_after_finetune.pt")


Epoch: 0
[Step=50]	Loss=0.9567	acc=0.6908	2569.5 examples/second
[Step=100]	Loss=0.7980	acc=0.7398	2986.1 examples/second
[Step=150]	Loss=0.7123	acc=0.7645	3195.0 examples/second
Test Loss=0.5566, Test acc=0.8184
Saving...

Epoch: 1
[Step=50]	Loss=0.4621	acc=0.8436	3465.2 examples/second
[Step=100]	Loss=0.4559	acc=0.8439	3003.9 examples/second
[Step=150]	Loss=0.4506	acc=0.8454	2968.8 examples/second
Test Loss=0.4842, Test acc=0.8402
Saving...

Epoch: 2
[Step=50]	Loss=0.4012	acc=0.8621	3590.0 examples/second
[Step=100]	Loss=0.3959	acc=0.8646	3654.9 examples/second
[Step=150]	Loss=0.3922	acc=0.8666	3297.8 examples/second
Test Loss=0.4540, Test acc=0.8508
Saving...

Epoch: 3
[Step=50]	Loss=0.3661	acc=0.8745	3229.8 examples/second
[Step=100]	Loss=0.3677	acc=0.8758	3722.0 examples/second
[Step=150]	Loss=0.3621	acc=0.8774	3225.1 examples/second
Test Loss=0.4344, Test acc=0.8585
Saving...

Epoch: 4
[Step=50]	Loss=0.3523	acc=0.8811	3395.3 examples/second
[Step=100]	Loss=0.3426	acc=0.8849	3495

In [ ]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("net_after_finetune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        out = layer.weight
        weight = out.cpu().detach().numpy()
        weight_re = weight.reshape(-1,1)

        weight_abs = np.abs(weight_re)
        N = weight_abs.shape[0]

        # Count number of zeros
        zeros = N- np.count_nonzero(weight_abs)
        # Count number of parameters
        total = weight_abs.shape[0]
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

### Lab2 (d) Iterative pruning

In [11]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    
    net.train()
    if epoch<10:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Increase model sparsity
                q = 8 *(epoch+1)
                prune_by_percentage(layer, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0470	acc=0.9857	3439.6 examples/second
[Step=100]	Loss=0.0469	acc=0.9861	3785.6 examples/second
[Step=150]	Loss=0.0471	acc=0.9856	3610.0 examples/second
Test Loss=0.3237, Test acc=0.9147

Epoch: 1
[Step=50]	Loss=0.0531	acc=0.9821	3574.8 examples/second
[Step=100]	Loss=0.0509	acc=0.9839	3663.5 examples/second
[Step=150]	Loss=0.0502	acc=0.9841	3629.7 examples/second
Test Loss=0.3241, Test acc=0.9151

Epoch: 2
[Step=50]	Loss=0.0542	acc=0.9819	3483.7 examples/second
[Step=100]	Loss=0.0526	acc=0.9827	3521.0 examples/second
[Step=150]	Loss=0.0510	acc=0.9835	3647.3 examples/second
Test Loss=0.3287, Test acc=0.9133

Epoch: 3
[Step=50]	Loss=0.0610	acc=0.9788	3126.9 examples/second
[Step=100]	Loss=0.0586	acc=0.9804	3726.5 examples/second
[Step=150]	Loss=0.0580	acc=0.9804	3692.8 examples/second
Test Loss=0.3347, Test acc=0.9121

Epoch: 4
[Step=50]	Loss=0.0717	acc=0.9759	3417.9 examples/second
[Step=100]	Loss=0.0707	acc=0.9761	3460.9 examples/second
[Step=150]	Loss=0.068

In [17]:
# Check sparsity of the final model, make sure it's 80%
net.load_state_dict(torch.load("net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here: can copy from previous question
        out = layer.weight
        weight = out.cpu().detach().numpy()
        weight_re = weight.reshape(-1,1)

        weight_abs = np.abs(weight_re)
        N = weight_abs.shape[0]

        # Count number of zeros
        zeros = N- np.count_nonzero(weight_abs)
        # Count number of parameters
        total = weight_abs.shape[0]
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
        
test(net)

Sparsity of head_conv.0.conv: 0.8009259259259259
Sparsity of body_op.0.conv1.0.conv: 0.8003472222222222
Sparsity of body_op.0.conv2.0.conv: 0.8003472222222222
Sparsity of body_op.1.conv1.0.conv: 0.8003472222222222
Sparsity of body_op.1.conv2.0.conv: 0.8003472222222222
Sparsity of body_op.2.conv1.0.conv: 0.8003472222222222
Sparsity of body_op.2.conv2.0.conv: 0.8003472222222222
Sparsity of body_op.3.conv1.0.conv: 0.8001302083333334
Sparsity of body_op.3.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.6.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.6.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.7.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.7.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.8.conv1.0.conv: 0.8000217013888888
Sparsity of body

### Lab2 (e) Global iterative pruning

In [10]:
def global_prune_by_percentage(net, q=80.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            out = layer.weight
            weight = out.cpu().detach().numpy()
            size=weight.shape
            weight_re = np.abs(weight).reshape(-1,1)
            
            flattened_weights.append(weight_re)
            
            # Flatten the weight and append to flattened_weights
    
    # Concate all weights into a np array
    
    
    flattened_weights= np.concatenate(flattened_weights)
    # Find global pruning threshold

    thres = np.percentile(flattened_weights,q)
    print(thres)

    
    # Apply pruning threshold to all layers
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            out = layer.weight
            weight = out.cpu().detach().numpy()
            weight_abs = np.abs(weight)
            size=weight_abs.shape
            
            # Generate a binary mask same shape as weight to decide which element to prune
            mask = np.where(weight_abs<thres,0,1)
       
            # Convert mask to torch tensor and put on GPU            
            mask = torch.from_numpy(mask).to(layer.weight.device)

    
            # Multiply the weight by mask to perform pruning
            output = torch.mul(mask, out)
            layer.weight.data = output#.float()

            

In [59]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    q=(epoch+1)*8
    
    net.train()
    # Increase model sparsity
    if epoch<10:
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_global_iterative_prune.pt")


Epoch: 0
0.0085436649620533
[Step=50]	Loss=0.0502	acc=0.9848	2732.0 examples/second
[Step=100]	Loss=0.0490	acc=0.9851	3805.0 examples/second
[Step=150]	Loss=0.0480	acc=0.9848	3532.1 examples/second
Test Loss=0.3246, Test acc=0.9143

Epoch: 1
0.01718289256095886
[Step=50]	Loss=0.0487	acc=0.9846	3008.5 examples/second
[Step=100]	Loss=0.0483	acc=0.9847	3893.2 examples/second
[Step=150]	Loss=0.0493	acc=0.9839	3513.1 examples/second
Test Loss=0.3235, Test acc=0.9157

Epoch: 2
0.026158433780074118
[Step=50]	Loss=0.0514	acc=0.9826	3465.8 examples/second
[Step=100]	Loss=0.0502	acc=0.9834	3779.5 examples/second
[Step=150]	Loss=0.0501	acc=0.9834	3833.5 examples/second
Test Loss=0.3250, Test acc=0.9152

Epoch: 3
0.03560339510440826
[Step=50]	Loss=0.0536	acc=0.9824	3448.3 examples/second
[Step=100]	Loss=0.0546	acc=0.9820	3336.2 examples/second
[Step=150]	Loss=0.0539	acc=0.9822	3160.9 examples/second
Test Loss=0.3251, Test acc=0.9128

Epoch: 4
0.04540476202964783
[Step=50]	Loss=0.0594	acc=0.9802	3

In [8]:
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        out = layer.weight
        np_weight = out.cpu().detach().numpy()
        N = np_weight.size
        # Count number of zeros
        zeros = N-np.count_nonzero(np_weight)
        # Count number of parameters
        total = N
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.3101851851851852
Sparsity of body_op.0.conv1.0.conv: 0.6571180555555556
Sparsity of body_op.0.conv2.0.conv: 0.6397569444444444
Sparsity of body_op.1.conv1.0.conv: 0.6258680555555556
Sparsity of body_op.1.conv2.0.conv: 0.6467013888888888
Sparsity of body_op.2.conv1.0.conv: 0.6315104166666666
Sparsity of body_op.2.conv2.0.conv: 0.6697048611111112
Sparsity of body_op.3.conv1.0.conv: 0.6254340277777778
Sparsity of body_op.3.conv2.0.conv: 0.6886935763888888
Sparsity of body_op.4.conv1.0.conv: 0.7252604166666666
Sparsity of body_op.4.conv2.0.conv: 0.7822265625
Sparsity of body_op.5.conv1.0.conv: 0.7248263888888888
Sparsity of body_op.5.conv2.0.conv: 0.8127170138888888
Sparsity of body_op.6.conv1.0.conv: 0.7322591145833334
Sparsity of body_op.6.conv2.0.conv: 0.7645399305555556
Sparsity of body_op.7.conv1.0.conv: 0.7768825954861112
Sparsity of body_op.7.conv2.0.conv: 0.826416015625
Sparsity of body_op.8.conv1.0.conv: 0.8526204427083334
Sparsity of body_op.8.conv

### Lab 3 (b) and (c): Fixed-point quantization

In [13]:
# Define quantized model and load weight
Nbits = 4 #Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)


Files already downloaded and verified
Test Loss=0.3861, Test accuracy=0.8972


In [14]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)   

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=0.0631	acc=0.9791	1160.8 examples/second
[Step=100]	Loss=0.0627	acc=0.9793	1084.8 examples/second
[Step=150]	Loss=0.0635	acc=0.9790	1087.4 examples/second
Test Loss=0.3351, Test acc=0.9090
Saving...

Epoch: 1
[Step=200]	Loss=0.0610	acc=0.9795	486.5 examples/second
[Step=250]	Loss=0.0657	acc=0.9776	1208.4 examples/second
[Step=300]	Loss=0.0643	acc=0.9780	1091.5 examples/second
[Step=350]	Loss=0.0638	acc=0.9778	1107.5 examples/second
Test Loss=0.3328, Test acc=0.9115
Saving...

Epoch: 2
[Step=400]	Loss=0.0627	acc=0.9775	462.1 examples/second
[Step=450]	Loss=0.0626	acc=0.9792	1162.2 examples/second
[Step=500]	Loss=0.0620	acc=0.9788	1153.1 examples/second
[Step=550]	Loss=0.0604	acc=0.9793	1036.6 examples/second
Test Loss=0.3312, Test acc=0.9116
Saving...

Epoch: 3
[Step=600]	Loss=0.0604	acc=0.9824	470.0 examples/second
[Step=650]	Loss=0.0618	acc=0.9800	913.4 examples/se

In [25]:
for Nbits in range(2,7):
    print("-"*20,"Nbits = ",Nbits,"-"*80)
    net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
    net = net.to(device)
    net.load_state_dict(torch.load("pretrained_model.pt"))
    test(net)
    # Quantized model finetuning
    finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)   

    # Load the model with best accuracy
    net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
    test(net)


-------------------- Nbits =  2 --------------------------------------------------------------------------------
Files already downloaded and verified
Test Loss=9.5441, Test accuracy=0.0899
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=1.3362	acc=0.6270	1361.4 examples/second
[Step=100]	Loss=1.0766	acc=0.6844	1145.8 examples/second
[Step=150]	Loss=0.9446	acc=0.7149	1324.1 examples/second
Test Loss=0.7871, Test acc=0.7617
Saving...

Epoch: 1
[Step=200]	Loss=0.5654	acc=0.8135	520.8 examples/second
[Step=250]	Loss=0.5645	acc=0.8120	1224.2 examples/second
[Step=300]	Loss=0.5453	acc=0.8176	1113.2 examples/second
[Step=350]	Loss=0.5343	acc=0.8194	1096.9 examples/second
Test Loss=0.6190, Test acc=0.8047
Saving...

Epoch: 2
[Step=400]	Loss=0.4880	acc=0.8330	527.8 examples/second
[Step=450]	Loss=0.4867	acc=0.8337	1189.1 examples/second
[Step=500]	Loss=0.4824	acc=0.8361	1124.4 examples/second
[Step=550]	Loss=0.4695	acc=0

In [7]:
for Nbits in range(2,5):
    print("-"*20,"Nbits = ",Nbits,"-"*80)


    net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
    net = net.to(device)
    net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
    test(net)
    # Quantized model finetuning
    finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

    # Load the model with best accuracy
    net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
    test(net)


-------------------- Nbits =  2 --------------------------------------------------------------------------------
Files already downloaded and verified
Test Loss=7.5443, Test accuracy=0.1085
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=2.3061	acc=0.1420	1386.4 examples/second
[Step=100]	Loss=2.2496	acc=0.1536	1291.0 examples/second
[Step=150]	Loss=2.2221	acc=0.1609	1179.1 examples/second
Test Loss=2.1494, Test acc=0.1864
Saving...

Epoch: 1
[Step=200]	Loss=2.1508	acc=0.1904	581.9 examples/second
[Step=250]	Loss=2.1455	acc=0.1874	1267.6 examples/second
[Step=300]	Loss=2.1379	acc=0.1929	1365.6 examples/second
[Step=350]	Loss=2.1300	acc=0.2001	1109.8 examples/second
Test Loss=2.1152, Test acc=0.2111
Saving...

Epoch: 2
[Step=400]	Loss=2.1082	acc=0.2305	578.6 examples/second
[Step=450]	Loss=2.1042	acc=0.2208	1192.5 examples/second
[Step=500]	Loss=2.1029	acc=0.2223	1086.6 examples/second
[Step=550]	Loss=2.0981	acc=0

### Lab3 (d) Quantize pruned model

In [9]:
# Define quantized model and load weight
Nbits = 6 #Change this value to finish (d)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net)


Files already downloaded and verified
Test Loss=0.3474, Test accuracy=0.8844


In [10]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0


KeyboardInterrupt: 

In [11]:
# FP_layers have been changed
for Nbits in range(2,5):
    print("-"*20,"Nbits = ",Nbits,"-"*80)


    net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
    net = net.to(device)
    net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
    test(net)
    # Quantized model finetuning
    finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

    # Load the model with best accuracy
    net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
    test(net)

-------------------- Nbits =  2 --------------------------------------------------------------------------------
Files already downloaded and verified
Test Loss=1530.5690, Test accuracy=0.1000
==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=2.2840	acc=0.1534	1268.5 examples/second
[Step=100]	Loss=2.2402	acc=0.1596	1137.2 examples/second
[Step=150]	Loss=2.2143	acc=0.1647	1195.1 examples/second
Test Loss=2.1660, Test acc=0.1815
Saving...

Epoch: 1
[Step=200]	Loss=2.1573	acc=0.1855	508.1 examples/second
[Step=250]	Loss=2.1320	acc=0.1934	1274.9 examples/second
[Step=300]	Loss=2.1276	acc=0.1966	1174.6 examples/second
[Step=350]	Loss=2.1227	acc=0.2019	1156.0 examples/second
Test Loss=2.1152, Test acc=0.2104
Saving...

Epoch: 2
[Step=400]	Loss=2.1142	acc=0.2114	481.8 examples/second
[Step=450]	Loss=2.0971	acc=0.2214	1129.2 examples/second
[Step=500]	Loss=2.0952	acc=0.2260	1234.2 examples/second
[Step=550]	Loss=2.0917	ac

### Lab3 (e) Prune quantized model

In [13]:
net = ResNetCIFAR(num_layers=20, Nbits=4)
net = net.to(device)

net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    q=(epoch+1)*8
    
    net.train()
    # Increase model sparsity
    if epoch<10:
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "quantized_net_after_global_iterative_prune.pt")


Epoch: 0
0.0085436649620533
[Step=50]	Loss=0.0697	acc=0.9747	715.1 examples/second
[Step=100]	Loss=0.0684	acc=0.9755	898.4 examples/second
[Step=150]	Loss=0.0697	acc=0.9753	922.9 examples/second
Test Loss=0.3410, Test acc=0.9088

Epoch: 1
0.017190853133797646
[Step=50]	Loss=0.0667	acc=0.9771	911.7 examples/second
[Step=100]	Loss=0.0700	acc=0.9756	829.2 examples/second
[Step=150]	Loss=0.0691	acc=0.9761	820.4 examples/second
Test Loss=0.3478, Test acc=0.9070

Epoch: 2
0.026171804219484326
[Step=50]	Loss=0.0737	acc=0.9742	851.1 examples/second
[Step=100]	Loss=0.0720	acc=0.9749	860.0 examples/second
[Step=150]	Loss=0.0715	acc=0.9754	905.3 examples/second
Test Loss=0.3462, Test acc=0.9070

Epoch: 3
0.03563041388988495
[Step=50]	Loss=0.0753	acc=0.9738	834.6 examples/second
[Step=100]	Loss=0.0741	acc=0.9736	861.0 examples/second
[Step=150]	Loss=0.0744	acc=0.9732	891.0 examples/second
Test Loss=0.3461, Test acc=0.9057

Epoch: 4
0.045436009764671326
[Step=50]	Loss=0.0848	acc=0.9718	919.2 examp

In [14]:
net.load_state_dict(torch.load("quantized_net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        np_weight = layer.weight.data.cpu().numpy()
        zeros = (np_weight==0).sum()
        total = np_weight.flatten().shape[0]
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.30787037037037035
Sparsity of body_op.0.conv1.0.conv: 0.6575520833333334
Sparsity of body_op.0.conv2.0.conv: 0.6397569444444444
Sparsity of body_op.1.conv1.0.conv: 0.6280381944444444
Sparsity of body_op.1.conv2.0.conv: 0.6488715277777778
Sparsity of body_op.2.conv1.0.conv: 0.6315104166666666
Sparsity of body_op.2.conv2.0.conv: 0.6684027777777778
Sparsity of body_op.3.conv1.0.conv: 0.6252170138888888
Sparsity of body_op.3.conv2.0.conv: 0.6882595486111112
Sparsity of body_op.4.conv1.0.conv: 0.7261284722222222
Sparsity of body_op.4.conv2.0.conv: 0.7820095486111112
Sparsity of body_op.5.conv1.0.conv: 0.7249348958333334
Sparsity of body_op.5.conv2.0.conv: 0.8132595486111112
Sparsity of body_op.6.conv1.0.conv: 0.7331814236111112
Sparsity of body_op.6.conv2.0.conv: 0.7640245225694444
Sparsity of body_op.7.conv1.0.conv: 0.7770453559027778
Sparsity of body_op.7.conv2.0.conv: 0.8261176215277778
Sparsity of body_op.8.conv1.0.conv: 0.8527018229166666
Sparsity of bod